In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from sklearn.metrics import confusion_matrix
import seaborn as sns

from sklearn.utils.class_weight import compute_class_weight

In [ ]:
# Importação das bibliotecas necessárias
import cv2
import random
from tensorflow.keras.applications.resnet50 import preprocess_input

# Função para modificar o matiz (hue) da imagem e realizar pré processamento
def modify_hue(image):
    image_hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    hue_shift = random.uniform(-0.1, 0.1)
    image_hsv[:,:,0] += hue_shift
    image_hsv[:,:,0] = np.clip(image_hsv[:,:,0], 0, 1)
    image_rgb = cv2.cvtColor(image_hsv, cv2.COLOR_HSV2RGB)
    return image_rgb

def preprocess_image(image):
    modified_image = modify_hue(image)
    standardized_image = preprocess_input(modified_image)
    return standardized_image

# Criação do gerador de dados para o treinamento
train_datagen = ImageDataGenerator(
    rotation_range=0,  # Ângulo de rotação máximo das imagens
    width_shift_range=0.1,  # Deslocamento máximo horizontal das imagens
    height_shift_range=0.1,  # Deslocamento máximo vertical das imagens
    shear_range=0.1,  # Valor máximo de cisalhamento das imagens
    zoom_range=0.1,  # Faixa de zoom aplicada às imagens
    brightness_range=[0.7, 1.3],  # Faixa de brilho aplicada às imagens
    fill_mode='nearest',  # Modo de preenchimento de pixels para transformações geométricas
    preprocessing_function=preprocess_image,  # Função de pré-processamento aplicada a cada imagem
)
 
# Criação do gerador de lotes de treinamento
train_generator = train_datagen.flow_from_directory(
    '/content/drive/Shareddrives/IA901/dados melanoma/data/interim/treino',  # Diretório das imagens de treinamento
    target_size=(224, 224),  # Tamanho das imagens após o redimensionamento
    batch_size=64,  # Tamanho do lote (batch size)
    class_mode='categorical'  # Modo de classificação das imagens
)

# Criação do gerador de dados para a validação
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=0,  # Ângulo de rotação máximo das imagens
    width_shift_range=0.1,  # Deslocamento máximo horizontal das imagens
    height_shift_range=0.1,  # Deslocamento máximo vertical das imagens
    shear_range=0.1,  # Valor máximo de cisalhamento das imagens
    zoom_range=0.1,  # Faixa de zoom aplicada às imagens
    brightness_range=[0.7, 1.3],  # Faixa de brilho aplicada às imagens
    fill_mode='nearest',  # Modo de preenchimento de pixels para transformações geométricas
    preprocessing_function=modify_hue  # Função de pré-processamento aplicada a cada imagem
)

# Criação do gerador de lotes de validação
validation_generator = test_datagen.flow_from_directory(
    '/content/drive/Shareddrives/IA901/dados melanoma/data/interim/validacao',  # Diretório das imagens de validação
    target_size=(224, 224),  # Tamanho das imagens após o redimensionamento
    batch_size=64,  # Tamanho do lote (batch size)
    class_mode='categorical'  # Modo de classificação das imagens
)

# Visualização das transformações de aumento de dados em uma imagem de exemplo
for i in range(1):
    image = train_generator[0][0][0].astype('uint8')
    plt.imshow(tf.keras.preprocessing.image.array_to_img(image))
    plt.show()


In [ ]:
# Calculamos os pesos das classes para que seja penalisado no treino posteriormente
classes = np.unique(train_generator.classes)
class_counts = np.bincount(train_generator.classes)
total_samples = np.sum(class_counts)
class_weights = total_samples / (len(classes) * class_counts)

train_class_weights = dict(enumerate(class_weights))
print(train_class_weights)

In [ ]:
# Definir o otimizador SGD
sgd = tf.keras.optimizers.legacy.SGD(learning_rate=0.001, momentum=0.9, decay=0.01, nesterov=True)
# learning_rate: Taxa de aprendizado
# momentum: Taxa de momento para acelerar a convergência
# decay: Taxa de decaimento para ajustar a taxa de aprendizado ao longo do tempo
# nesterov: Indica se deve ser usado o método Nesterov para o cálculo do gradiente

# Definir o modelo ResNet50
model_ResNet50 = tf.keras.Sequential([
    tf.keras.applications.ResNet50(
        input_shape=(224, 224, 3),  # Formato da entrada
        weights='imagenet'  # Usar pesos pré-treinados da ImageNet
    ),
    tf.keras.layers.Reshape((-1, 1, 1000)),  # Remodelar para um tensor 4-dimensional
    tf.keras.layers.GlobalAveragePooling2D(),  # Camada de pooling global para reduzir a dimensionalidade
    tf.keras.layers.Dense(2, activation='sigmoid')  # Camada densa com ativação sigmoid para classificação binária
])

# Compilar o modelo
model_ResNet50.compile(
    optimizer=sgd, 
    metrics=['AUC'],  
    loss='binary_crossentropy'  
)

In [ ]:
# Congelar os pesos da primeira camada
model_ResNet50.layers[0].trainable = False

# Percorrer as camadas até a quinta camada e congelar seus pesos
for layer in model_ResNet50.layers[:5]:
    layer.trainable = False

# Exibir um resumo do modelo
model_ResNet50.summary()

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Reshape, GlobalAveragePooling2D, Dense
from tensorflow.keras.callbacks import Callback

# Configurando o modo de execução do TensorFlow para eager execution (execução imediata)
tf.config.run_functions_eagerly(True)  # Caso contrário, pode ocorrer um erro

# Callbacks
cb_early_stopper = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
cb_checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath='/content/drive/Shareddrives/IA901/dados melanoma/models/', monitor='val_loss', save_best_only=True)

callbacks_list = [cb_checkpointer, cb_early_stopper]

# Arquitetura do modelo
base_model = tf.keras.applications.ResNet50(
    include_top=False,  # Excluir a camada fully-connected no topo
    input_shape=(224, 224, 3)  # Formato da entrada
)
x = base_model.output
x = tf.keras.layers.Reshape((-1, 1, 2048))(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(2, activation='sigmoid')(x)
model = tf.keras.Model(inputs=base_model.input, outputs=outputs)

# Compilando o modelo
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['AUC'])

# Treinamento
history = model.fit(
    train_generator,  
    epochs=20,  
    validation_data=validation_generator, 
    callbacks=callbacks_list,
    class_weight=train_class_weights
)


In [ ]:
# Salvar o histórico de treinamento em um arquivo separado
history_path = '/content/drive/Shareddrives/IA901/dados melanoma/models/history_resnet50.pickle'
with open(history_path, 'wb') as f:
    pickle.dump(history.history, f)

In [ ]:
#from tensorflow.keras.models import load_model

#model_path = '/content/drive/Shareddrives/IA901/dados melanoma/models/'
#history = load_model(model_path)

In [ ]:
# Curvas de erro e AUC para treino e validação
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Treino')
plt.plot(history.history['val_loss'], label='Validação')
plt.xlabel('Época')
plt.ylabel('Erro')
plt.title('Erro de Treinamento e de Validação')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['auc'], label='Treino')
plt.plot(history.history['val_auc'], label='Validação')
plt.xlabel('Época')
plt.ylabel('AUC')
plt.title('AUC de Treinamento e de Validação')
plt.legend()
plt.show()


In [ ]:
# Prediçoes e matriz de confusão
predictions = model.predict(validation_generator)
y_true = validation_generator.classes
y_pred = predictions.argmax(axis=1)

confusion_mat = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(confusion_mat)

false_positive_rate = confusion_mat[0, 1] / (confusion_mat[0, 1] + confusion_mat[0, 0])
print("False Positive Rate:", false_positive_rate)

In [ ]:
# Obtém as classes reais e as classes previstas
y_true = validation_generator.classes
y_pred = predictions.argmax(axis=1)

# Calcula a matriz de confusão
cm = confusion_matrix(y_true, y_pred)

# Plot da matriz de confusão
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.colorbar()

# Obtém os rótulos das classes
class_names = validation_generator.class_indices
class_names = list(class_names.keys())

tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names, rotation=45)
plt.yticks(tick_marks, class_names)

# Calculate the total numbers in each group
group_counts = ['{0:0.0f}'.format(value) for value in cm.flatten()]
group_percentages = ['{0:.2%}'.format(value) for value in cm.flatten() / np.sum(cm)]

# Add the total numbers to the plot
labels = [f"{v1}\n{v2}" for v1, v2 in zip(group_counts, group_percentages)]
labels = np.asarray(labels).reshape(cm.shape[0], cm.shape[1])

# Show the total numbers in each group
thresh = cm.max() / 2.
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, labels[i, j], horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

plt.xlabel('Predito')
plt.ylabel('Verdadeiro')
plt.title('Matriz de confusão')
plt.show()